In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('../unzippedData/im.db')

In [3]:
! ls ../unzippedData

bom.movie_gross.csv  rt.movie_info.tsv    tmdb.movies.csv
im.db                rt.reviews.tsv       tn.movie_budgets.csv


In [4]:
# Function for Cleaning Column headers

def clean_column_headers(dataframe):
    dataframe.columns = dataframe.columns.str.title()
    dataframe.columns = dataframe.columns.str.replace("_", " ")

In [15]:
# Function for cleaning comma seperated string values of a series into list of strings. Accepts one series.

def clean_comma_series(Series):
    Series = Series.str.replace("_", " ")
    Series = Series.str.title()
    Series = Series.str.split(",")
    return Series

In [6]:
cursor = conn.cursor()

![Image of ERD Diagram IMDB](https://raw.githubusercontent.com/learn-co-curriculum/dsc-phase-2-project-v3/main/movie_data_erd.jpeg)

In [7]:
# displaying table

imdb_query = """ 
    SELECT *
    FROM sqlite_master
"""
imdb_df = pd.read_sql(imdb_query, conn)
imdb_df

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [8]:
# Connecting to Persons Table

persons_query = """ 
    SELECT *
    FROM persons
"""

persons_df = pd.read_sql(persons_query, conn)

# Data Cleaning

clean_column_headers(persons_df)
persons_df['Primary Profession'] = clean_comma_series(persons_df['Primary Profession'])
persons_df.drop(persons_df[persons_df['Death Year'].isna()==False].index, inplace=True)
persons_df.drop(persons_df[persons_df['Birth Year'] < 1930].index, inplace=True)
persons_df

,Person Id,Primary Name,Birth Year,Death Year,Primary Profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"[Miscellaneous, Production Manager, Producer]"
1,nm0061865,Joseph Bauer,NaN,NaN,"[Composer, Music Department, Sound Department]"
2,nm0062070,Bruce Baum,NaN,NaN,"[Miscellaneous, Actor, Writer]"
3,nm0062195,Axel Baumann,NaN,NaN,"[Camera Department, Cinematographer, Art Depar..."
4,nm0062798,Pete Baxter,NaN,NaN,"[Production Designer, Art Department, Set Deco..."
...,...,...,...,...,...
606643,nm9990381,Susan Grobes,NaN,NaN,[Actress]
606644,nm9990690,Joo Yeon So,NaN,NaN,[Actress]
606645,nm9991320,Madeline Smith,NaN,NaN,[Actress]
606646,nm9991786,Michelle Modigliani,NaN,NaN,[Producer]


In [9]:
# Connecting to Principals Table

principals_query = """ 
    SELECT *
    FROM principals
"""

principals_df = pd.read_sql(principals_query, conn)
clean_column_headers(principals_df)

principals_df.head()

,Movie Id,Ordering,Person Id,Category,Job,Characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [10]:
# Connecting to Known_For Table

known_for_query = """ 
    SELECT *
    FROM known_for
"""

known_for_df = pd.read_sql(known_for_query, conn)
clean_column_headers(known_for_df)

known_for_df.head()

,Person Id,Movie Id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [16]:
# Join persons and known for then movie basics

persons_query = """ 
    SELECT *
    FROM persons
    JOIN known_for
        USING(person_id)
    JOIN movie_basics
        USING(movie_id)
"""

joined_df = pd.read_sql(persons_query, conn)


clean_column_headers(joined_df)
joined_df = pd.DataFrame(data=joined_df, columns=['Primary Name', 'Primary Title', 'Primary Profession'])
# joined_df['Primary Profession'] = clean_comma_series(persons_df['Primary Profession'])

joined_df

AttributeError: Can only use .str accessor with string values!

In [ ]:
# Connecting to Movie Basics Table

movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""

movie_basics_df = pd.read_sql(movie_basics_query, conn)
clean_column_headers(movie_basics_df)

# movie_basics_df[(movie_basics_df['Primary Title'].str.contains('Paranormal Activity'))]
movie_basics_df['Start Year'].value_counts()

In [ ]:
# Connecting to Movie Ratings Table

movie_ratings_query = """ 
    SELECT *
    FROM movie_ratings
"""
movie_ratings_df = pd.read_sql(movie_ratings_query, conn)
clean_column_headers(movie_ratings_df)

movie_ratings_df

In [ ]:
persons_df['Death Year'].value_counts()

In [ ]:
# dropping dead people. no pun intended
persons_df.drop(persons_df[persons_df['Death Year'].isna()==False].index, inplace=True)
persons_df

In [ ]:
# Connecting to Movie Akas Table

movie_akas_query = """ 
    SELECT *
    FROM movie_akas
"""
movie_akas_df = pd.read_sql(movie_akas_query, conn)
clean_column_headers(movie_akas_df)
movie_akas_df

In [ ]:
# Connecting to Movie Basics Table

movie_basics_query = """ 
    SELECT *
    FROM movie_basics
"""
movie_basics_df = pd.read_sql(movie_basics_query, conn)
clean_column_headers(movie_basics_df)
movie_basics_df['Genres'] = clean_comma_series(movie_basics_df['Genres'])
movie_basics_df
